<a href="https://colab.research.google.com/github/Shresth-Mehta/Data-Extraction/blob/master/Extracting_data_from_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install tesseract-ocr
!pip install pytesseract
!pip install pdf2image
!apt-get install poppler-utils

In [3]:
import pytesseract
from PIL import Image
from pdf2image import convert_from_path
import os
from tqdm import tqdm
import cv2
from pytesseract import Output
import numpy as np
import pandas as pd
from google.colab.patches import cv2_imshow

In [4]:
def pdf_to_img(input_file,folder_path):
  print("Converting pdf to jpeg ...")
  images = convert_from_path(input_file)
  with tqdm(total=len(images)) as pbar:
    for i,img in enumerate(images):
      path=os.path.join(folder_path,"img_"+str(i)+".jpeg")
      img.save(path, "JPEG", optimize=True, progressive=True)
      pbar.update(1)

def edge_detection(img):
  kernel_len=np.array(img).shape[1]//100
  horizontal_kernel_len=np.array(img).shape[0]//25
  ver_kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(1,kernel_len))
  hor_kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(horizontal_kernel_len,1))
  #kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
  img_1=cv2.erode(img,ver_kernel,iterations=3)
  vertical_lines=cv2.dilate(img_1,ver_kernel,iterations=3)
  img_2=cv2.erode(img,hor_kernel,iterations=3)
  horizontal_lines=cv2.dilate(img_2,hor_kernel,iterations=3)
  return vertical_lines,horizontal_lines

def get_combined_image(vertical_lines,horizontal_lines):
  kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
  img_vh=cv2.addWeighted(vertical_lines,0.5,horizontal_lines,0.5,0.0)
  img_vh=cv2.erode(~img_vh,kernel,iterations=2)
  thresh,img_vh=cv2.threshold(img_vh,175,255,cv2.THRESH_BINARY)
  return img_vh

In [6]:
#Make the required folders before executing
pdf_to_img("/content/Order Regarding Containment Zone dated 17062020_1.pdf","/content/gurugram")

Converting pdf to jpeg ...


100%|██████████| 9/9 [00:02<00:00,  3.96it/s]


In [8]:
path="/content/gurugram"
with tqdm(total=len(os.listdir(path))) as pbar:
  for i,img_path in enumerate(sorted(os.listdir(path))):
    img_path=os.path.join(path,img_path)
    img=cv2.imread(img_path,0)  #0 helps u to read it as greyscale instead of RGB
    
   
    thresh,img_th = cv2.threshold(img,185,255,cv2.THRESH_BINARY)
    img_bin = 255-img_th

    # finding vertical and horizontal images and combining them 
    vertical_lines,horizontal_lines=edge_detection(img_bin)
    combined_img=get_combined_image(vertical_lines,horizontal_lines)

    # removing table from the image
    img_th[combined_img==0]=255

    #Reducing noise in the image
    kernel=cv2.getStructuringElement(cv2.MORPH_RECT,(2,2))
    img_noise_rem=cv2.dilate(img_th,kernel,iterations=1)
    img_noise_rem=cv2.erode(img_noise_rem,kernel,iterations=1)
    thresh,img_denoised = cv2.threshold(img_noise_rem,175,255,cv2.THRESH_BINARY)

    # extraction of the data into a string
    custom_config=r'--oem 3 --psm 4 -1 eng'
    s=pytesseract.image_to_string(img_denoised,lang="eng",config=custom_config)
    l=s.split("\n")
    file_name=str(i)+"_sample.txt"
    #save_file_path=os.path.join(save_folder_path,file_name)
    with open(file_name,"w") as f:
      for i in l:
        f.write(i+"\n")
    pbar.update(1)

100%|██████████| 9/9 [01:03<00:00,  7.03s/it]
